# Appendix 10.1: Tokens/Vocabulary issues and Chaining Prompts

- [Lesson](#Lesson)
- [Example Playground](#Example-Playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
# Import python's built-in regular expression library
import re
from ollama import chat, ChatResponse, Options


# Stores the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME

def get_completion(messages: str, system_prompt=""):
    response = chat(
        model=MODEL_NAME,
        options=Options(
            max_tokens=2000,
            temperature=0.0,
            num_ctx=8192 # larger context defined to include large prompt in this chapter, default value is 2048
        ),
        messages=messages
    )
    return response.message.content

---

## Lesson

The saying goes, "Writing is rewriting." It turns out, **Ollama can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt Ollama to "think again". The ways that feel natural to ask a human to double check their work will also generally work for Ollama. Problem is that there are some limitations in each model that can make tasks like following impossible even with Prompt Chaining.

### Examples

In this example, we ask Ollama to come up with ten words... but one or more of them isn't a real word.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ar'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Ollama's response
first_response = get_completion(messages)
print(first_response)

**Asking Ollama should make its answer more accurate** but it depends on tokenization and vocabulary of the model used. Qwen 2.5 14B really struggles with this exercise.

Below, we've pulled down Ollama's incorrect response from above and added another turn to the conversation asking Ollama to fix its previous answer.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

But is Ollama revising its answer just because we told it to? What if we start off with a correct answer already? Will Ollama lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

You may notice that if you generate a response from the above block a few times, Ollama leaves the words as is most of the time, but still occasionally changes the words even though they're all already correct. What can we do to mitigate this? Per Chapter 8, we can give Ollama an out! Let's try this one more time.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

Try generating responses from the above code a few times to see that Ollama is much better at sticking to its guns now.

You can also use prompt chaining to **ask Ollama to make its responses better**. Below, we asked Ollama to first write a story, and then improve the story it wrote. Your personal tastes may vary, but many might agree that Ollama's second version is better.

First, let's generate Ollama's first version of the story.

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Ollama's response
first_response = get_completion(messages)
print(first_response)

Now let's have Ollama improve on its first draft.

In [ ]:
second_user = "Make the story better, like a best seller author would write it."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

This form of substitution is very powerful. We've been using substitution placeholders to pass in lists, words, Ollama's former responses, and so on. You can also **use substitution to do what we call "function calling," which is asking Ollama to perform some function, and then taking the results of that function and asking Ollama to do even more afterward with the results**. It works like any other substitution. More on this in the next appendix.

Below is one more example of taking the results of one call to Ollama and plugging it into another, longer call. Let's start with the first prompt (which includes prefilling Ollama's response this time).

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Ollama's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(first_response)

Let's pass this list of names into another prompt.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

Now that you've learned about prompt chaining, head over to Appendix 10.2 to learn how to implement function calling using prompt chaining.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Ollama's responses.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ar'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Ollama's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Ollama's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better, like a best seller author would write it."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Ollama's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Ollama's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Ollama's response -------------------------------------")
print(get_completion(messages))